In [34]:
import numpy as np
from PIL import ImageGrab
from PIL import Image
import cv2
import io; 
import time

In [112]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
game_url = "game/dino.html"
chrome_driver_path = "../chromedriver.exe"
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(200, 300)
        self._driver.get(os.path.abspath(game_url))
        if custom_config:
            self._driver.execute_script("Runner.config.ACCELERATION=0")
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        return self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
#     def grab_screen(self):
#         print("grab start")
#         image = self._driver.get_screenshot_as_png()
# #         performance_measure.send(time.time())
#         image = np.fromstring(image, np.uint8)
# #         performance_measure.send(time.time())
#         image = cv2.imdecode(image,cv2.IMREAD_UNCHANGED) 
# #         performance_measure.send(time.time())
#         image = process_img(image)
# #         performance_measure.send(time.time())
# #         cv2.imwrite('screenshot.png',image)
#         return image
    def end(self):
        self._driver.close()

In [113]:
class DinoAgent:
    def __init__(self,game):
        self._game = game;
        self.jump();
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
        time.sleep(0.5)
    def duck(self):
        self._game.press_down()

In [92]:
#processing image as required
def process_img(image):
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
#     image = image[10:140,0:200] #img[y:y+h, x:x+w]
#     image = resized_image = cv2.resize(image, (80, 80)) 
    image = cv2.resize(image, (0,0), fx = 0.25, fy = 0.20)
    image = image[2:42,10:90] #img[y:y+h, x:x+w]
    return  image

In [108]:
def grab_screen(_driver = None):
    screen =  np.array(ImageGrab.grab(bbox=(0,180,400,400)))
    image = process_img(screen)
    if _driver!=None:
        image = _driver.get_screenshot_as_png()
    return image

In [96]:
# # print(grab_screen().shape)
# game = Game()
# dino = DinoAgent(game)
# last_time = time.time()
# while(True):
    
# # #     print('loop took {} seconds'.format(time.time()-last_time))
# # #     last_time = time.time()
# # #     cv2.imwrite("./img_data/dino"+str(time())+".jpg",image)
# # #     dino.duck()
# #     #exit on q pres
# # #     print('{0} {1} '.format(r_t,end_t))
# # #     cv2.imshow('window',game.grab_screen())
    
#     image = grab_screen()
#     cv2.imshow('window',image)
#     print(image.shape[0])
# #     cv2.destroyAllWindows()

# # #     from matplotlib import pyplot as plt
# # #     plt.imshow(image)
# # #     plt.title('my picture')
# # #     plt.show()

# # #     grab_screen()
# #     if(dino.is_crashed()):
# #         #jumping starts the game again if dino has crashed
# # #         temp = (game.get_score())
# #         game.restart()
#     if (cv2.waitKey(25) & 0xFF == ord('q')):
#         cv2.destroyAllWindows()
#         game.end()
# #         cv2.imwrite('dino.jpg',image)
#         break

In [124]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 500 #10000. # timesteps to observe before training
EXPLORE = 3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 10
LEARNING_RATE = 1e-4

In [90]:
img_rows , img_cols = 80,40
img_channels = 4 #We stack 4 frames

In [33]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

In [118]:
def trainNetwork(model,game_state,observe=False):
    # open up a game state to communicate with emulator
    last_time = time.time()
    # store the previous observations in replay memory
    D = deque()
#     display = show_img()
#     display.__next__()
    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 
    x_t, r_0, terminal = game_state.get_state(do_nothing)
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    

    #In Keras, need to reshape
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4


    if observe :#args['mode'] == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON
#         model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = 0
    while (True):
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([ACTIONS])
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)
                action_index = max_Q
                a_t[action_index] = 1
        #We reduced the epsilon gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
#       display.send(x_t1)
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 80, 80, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                terminal = minibatch[i][4]
                # if terminated, only equals reward

                inputs[i:i + 1] = state_t    #I saved down s_t

                targets[i] = model.predict(state_t)  # Hitting each buttom probability
                Q_sa = model.predict(state_t1)

                if terminal:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            # targets2 = normalize(targets)
            loss += model.train_on_batch(inputs, targets)

        s_t = s_t1
        t = t + 1

        # save progress every 10000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [29]:
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    model = buildmodel()
    trainNetwork(model,game_state,observe=observe)

In [122]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img()
        self._display.__next__()
    def get_state(self,actions):
        
        reward = 0.1
        is_over = False
        if actions[1] == 1:
            self._agent.jump()
            reward = 0.1
            time.sleep(0.2)
        if self._agent.is_crashed():
            self._game.restart()
            reward = -1
            is_over = True
        
#         self._game.resume()
        image = grab_screen()#self._game.grab_screen()
#         self._game.pause()
        self._display.send(image)
        return image, reward, is_over

In [17]:
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf
from collections import deque
import random
import json

Using TensorFlow backend.


In [18]:
def show_img():
    """
    Show images in new window
    """
    frame = 0
    while True:
        screen = (yield)
        cv2.imshow("preview", screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

        frame += 1

In [19]:
def measure_performance():
    performance_time = time.time()
    last_time = performance_time
    while True:
        performance_time = yield 
        
        print('step took {} seconds'.format(time.time()-last_time))
        last_time = performance_time

In [125]:
playGame(observe=False)

Now we build the model
We finish building the model


C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(40, 80, 4..., strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  


loop took 1.1554372310638428 seconds
TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04964709281921387 seconds
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04808807373046875 seconds
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04950594902038574 seconds
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05154681205749512 seconds
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05414295196533203 seconds
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.048143625259399414 seconds
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05717134475708008 seconds
TIMESTEP 8 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop too

loop took 0.05417752265930176 seconds
TIMESTEP 70 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.7825241088867188 seconds
TIMESTEP 71 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04609394073486328 seconds
TIMESTEP 72 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05076146125793457 seconds
TIMESTEP 73 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04945802688598633 seconds
TIMESTEP 74 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04926347732543945 seconds
TIMESTEP 75 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05378460884094238 seconds
TIMESTEP 76 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04910087585449219 seconds
TIMESTEP 77 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
l

loop took 0.051252126693725586 seconds
TIMESTEP 136 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04913592338562012 seconds
TIMESTEP 137 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.05020737648010254 seconds
TIMESTEP 138 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.049863576889038086 seconds
TIMESTEP 139 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.07336544990539551 seconds
TIMESTEP 140 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.7737030982971191 seconds
TIMESTEP 141 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD -1 / Q_MAX  0 / Loss  0
loop took 0.05006742477416992 seconds
TIMESTEP 142 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.04947376251220703 seconds
TIMESTEP 143 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / 

...

TIMESTEP 11728 / STATE explore / EPSILON 0.09962614090007214 / ACTION 0 / REWARD 0.1 / Q_MAX  0.182255 / Loss  0.0221769735217
loop took 0.28878259658813477 seconds
TIMESTEP 11729 / STATE explore / EPSILON 0.09962610760007215 / ACTION 0 / REWARD 0.1 / Q_MAX  0.515451 / Loss  0.0101163061336
loop took 0.29993534088134766 seconds
TIMESTEP 11730 / STATE explore / EPSILON 0.09962607430007216 / ACTION 0 / REWARD 0.1 / Q_MAX  0.249818 / Loss  0.0148909687996
loop took 0.3109402656555176 seconds
TIMESTEP 11731 / STATE explore / EPSILON 0.09962604100007216 / ACTION 0 / REWARD 0.1 / Q_MAX  0.113142 / Loss  0.0121621582657
loop took 0.32138895988464355 seconds
TIMESTEP 11732 / STATE explore / EPSILON 0.09962600770007217 / ACTION 0 / REWARD 0.1 / Q_MAX  0.51554 / Loss  0.018685773015
loop took 0.2945132255554199 seconds
TIMESTEP 11733 / STATE explore / EPSILON 0.09962597440007218 / ACTION 0 / REWARD -1 / Q_MAX  0.555829 / Loss  0.0102226380259
loop took 0.2877631187438965 seconds
TIMESTEP 11734 /

TIMESTEP 11778 / STATE explore / EPSILON 0.09962447590007247 / ACTION 0 / REWARD 0.1 / Q_MAX  0.272882 / Loss  0.0220252443105
loop took 0.3002619743347168 seconds
TIMESTEP 11779 / STATE explore / EPSILON 0.09962444260007247 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.981139 / Loss  0.0243025887758
loop took 0.29793286323547363 seconds
TIMESTEP 11780 / STATE explore / EPSILON 0.09962440930007248 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.862473 / Loss  0.00705604255199
loop took 0.31487488746643066 seconds
TIMESTEP 11781 / STATE explore / EPSILON 0.09962437600007248 / ACTION 0 / REWARD 0.1 / Q_MAX  0.486467 / Loss  0.0247321482748
loop took 0.3071739673614502 seconds
TIMESTEP 11782 / STATE explore / EPSILON 0.09962434270007249 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.912129 / Loss  0.013371582143
loop took 0.32689404487609863 seconds
TIMESTEP 11783 / STATE explore / EPSILON 0.0996243094000725 / ACTION 0 / REWARD -1 / Q_MAX  0.523216 / Loss  0.017275871709
loop took 0.32266831398010254 seconds
TIMESTEP 117

TIMESTEP 11828 / STATE explore / EPSILON 0.09962281090007279 / ACTION 0 / REWARD 0.1 / Q_MAX  0.419867 / Loss  0.00814326573163
loop took 0.29937267303466797 seconds
TIMESTEP 11829 / STATE explore / EPSILON 0.09962277760007279 / ACTION 0 / REWARD 0.1 / Q_MAX  0.439954 / Loss  0.0084927175194
loop took 0.30462121963500977 seconds
TIMESTEP 11830 / STATE explore / EPSILON 0.0996227443000728 / ACTION 0 / REWARD 0.1 / Q_MAX  0.297854 / Loss  0.0337952598929
loop took 1.0217196941375732 seconds
TIMESTEP 11831 / STATE explore / EPSILON 0.0996227110000728 / ACTION 1 / REWARD 0.1 / Q_MAX  0.133778 / Loss  0.00622768281028
loop took 0.2775914669036865 seconds
TIMESTEP 11832 / STATE explore / EPSILON 0.09962267770007281 / ACTION 0 / REWARD 0.1 / Q_MAX  0.142701 / Loss  0.00680278101936
loop took 0.2818019390106201 seconds
TIMESTEP 11833 / STATE explore / EPSILON 0.09962264440007282 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.375649 / Loss  0.00823467131704
loop took 0.3066740036010742 seconds
TIMESTEP 11

TIMESTEP 11878 / STATE explore / EPSILON 0.09962114590007311 / ACTION 0 / REWARD -1 / Q_MAX  0.400044 / Loss  0.00875490903854
loop took 0.34048008918762207 seconds
TIMESTEP 11879 / STATE explore / EPSILON 0.09962111260007311 / ACTION 0 / REWARD 0.1 / Q_MAX  0.237202 / Loss  0.00900947675109
loop took 0.3669753074645996 seconds
TIMESTEP 11880 / STATE explore / EPSILON 0.09962107930007312 / ACTION 0 / REWARD 0.1 / Q_MAX  0.240835 / Loss  0.0161443427205
loop took 0.3699667453765869 seconds
TIMESTEP 11881 / STATE explore / EPSILON 0.09962104600007313 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.200969 / Loss  0.0115622617304
loop took 0.2871057987213135 seconds
TIMESTEP 11882 / STATE explore / EPSILON 0.09962101270007313 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0376615 / Loss  0.0127065209672
loop took 0.3145134449005127 seconds
TIMESTEP 11883 / STATE explore / EPSILON 0.09962097940007314 / ACTION 0 / REWARD 0.1 / Q_MAX  0.238067 / Loss  0.0130870044231
loop took 0.28577494621276855 seconds
TIMESTEP 11

StopIteration: 

In [ ]:
# # import cv2
# # image = cv2.imread('screenshot.png')
# # image = image[10:110,0:200] #img[y:y+h, x:x+w]
# from matplotlib import pyplot as plt
# plt.imshow(image)
# plt.title('my picture')
# plt.show()